In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# GUI - Graphical User Interface / Benutzeroberfläche
In diesem Abschnitt soll die graphische Oberfläche des Spiels entwickelt werden. Dies beinhaltet die Interaktion des Spielers mit dem Spielaufbau.

## Mögliche Positionen
Die Grafik unten zeigt die Notation für die Position der Spielsteine. Die Positionen werden als Tupel bestehend aus Ringnummer und Zellnummer beschrieben. Die Ringe, in der Abbildung farbig markiert, sind dabei von außen beginnend mit 0, 1 und 2 nummeriert. Für die Bezeichnung der Zellen wird in jedem Ring oben links mit 0 begonnen und im Uhrzeigersinn bis 7 weitergezählt. So ergibt sich eine eindeutige Notation der Form `(ring, cell)`.

<img src="board_positions.png" alt="Mögliche Positionen" width="800"/>

## Definitionen
Zunächst werden einige Definitionen für die gesamte Implementation festgesetzt: 
- `ring` = eines der 3 Quadrate (Wertebereich: 0-2)
- `cell` = ein Punkt auf einem Ring (Wertebereich: 0-7)
- `position` = Tupel: (ring, cell)
- `canvas` = Liste aus 3 Listen, die angeben, ob, und wenn ja welcher Stein dort sitzt. Dabei symbolisiert jede Liste einen Ring. In jeder Liste sind neun Werte. Je einer für eine Zelle im Ring. Eine 0 sagt dabei aus, dass an der entsprechenden Position kein Stein ist. Eine 1 oder 2 stehen für einen Stein des zugehörigen Spielers.
- `remaining` = \[Anzahl noch nicht gesetzter Steine Spieler 1, Anzahl noch nicht gesetzte Steine Spieler 2\]
- `state` = \[remaining, board\]


## Importe
Zur Visualisierung des Spielbrettes wird [ipycanvas](https://ipycanvas.readthedocs.io/en/latest/index.html) verwendet. Die Statusanzeigen sind Widgets der [ipywidget-Library](https://ipywidgets.readthedocs.io/en/latest/index.html).

In [ ]:
%run ./Muehle_Logic.ipynb
%run ./Muehle_Algo.ipynb
import ipycanvas
from ipycanvas import MultiCanvas

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import clear_output

## Konstanten

Um den Quellcode besser les- und wartbar zu machen, werden einige Konstanten definiert.
Für das Aussehen des Spielfelds:
- `BOARD_SIZE` = Größe des Spielfeldes in Pixeln
- `DOT_RADIUS` = Radius der schwarzen kleinen Punkte, die mögliche Positionen markieren (in Abhängigkeit von der Spielfeldgröße)
- `PIECE_RADIUS` = Radius der Spielsteine
- `COLOUR` = Farben der [dots, pieces_player_1, pieces_player_2]
- `COLOUR_HINT` = Farbe, in der mögliche Zielfelder markiert werden
- `COLOUR_OPPONENT` = Farbe, in der entfernbare Steine markiert werden
- `PADDING` = relativer Abstand des äußersten Quadrats zum Spielfeldrand
- `DISTANCE` = relativer Abstand zwischen den Quadraten des Spielfelds
- `TRANSPARENCY_DEFAULT` = Standardwert für die Transparenz
- `TRANSPARENCY_HINT` = Transparenz der Kreise möglicher Zielfelder und entfernbarer Steine

Für die Funktion des Spielfelds:
- `PLAYER` = Spieler
- `PLAYER` = Spielsteinfarben der Spieler
- `START_STATE` = Startzustand des Spielfelds, beide Spieler haben noch keinen Stein gesetzt und das Spielfeld ist leer

In [ ]:
# Aussehen
BOARD_SIZE      = 400
DOT_RADIUS      = BOARD_SIZE * 0.025
PIECE_RADIUS    = BOARD_SIZE * 0.04
COLOUR          = ['black', 'white', 'sienna']
COLOUR_HINT     = 'green'
COLOUR_OPPONENT = 'red'
PADDING         = 0.05
DISTANCE        = 0.15
TRANSPARENCY_DEFAULT = 1.0
TRANSPARENCY_HINT    = 0.5

# Spiel
PLAYER          = [1, 2]
PLAYER_COLOUR   = ['Weiß', 'Braun']
START_STATE     = [[9, 9], [          # Anzahl zu setzender Steine (Spieler_1 (Weiß), Spieler_2 (Braun))
                  [0, 0, 0, 0, 0, 0, 0, 0], # ring 0
                  [0, 0, 0, 0, 0, 0, 0, 0], # ring 1
                  [0, 0, 0, 0, 0, 0, 0, 0]  # ring 2
                  ]]    

 Die Funktion `row()` liefert einen relativen Wert für die Position der eingegebenen Reihe. `col()` macht dies für die Spalten. Der Eingabeparameter `number` gibt dabei an, die wie vielte Zeile von oben bzw. die wie vielte Spalte von links ausgegeben werden soll. Der Rückgabewert ist eine rationale Zahl zwischen 0,05 und 0,95. Multipliziert mit der Spielfeldbreite bzw. -höhe ergibt sich die absolute Position der Ecken und Schnittpunkte der Spielfeldlinien.

In [ ]:
#Reihe von oben nach unten (0-6)
def row(number):
    return PADDING + DISTANCE * number
#Spalte von links nach rechts (0-6)
def col(number):
    return row(number)

`POSITIONS` beschreibt die möglichen Punkte, an denen Steine sitzen können. Der Aufruf der Positionen erfolgt gemäß der oben vorgestellten Notation im Format `POSITIONS[ring][cell]`.

In [ ]:
POSITIONS = [([col(0), row(0)], [col(3), row(0)], [col(6), row(0)], [col(6), row(3)], [col(6), row(6)], [col(3), row(6)], [col(0), row(6)], [col(0), row(3)]), #ring 0
             ([col(1), row(1)], [col(3), row(1)], [col(5), row(1)], [col(5), row(3)], [col(5), row(5)], [col(3), row(5)], [col(1), row(5)], [col(1), row(3)]), #ring 1
             ([col(2), row(2)], [col(3), row(2)], [col(4), row(2)], [col(4), row(3)], [col(4), row(4)], [col(3), row(4)], [col(2), row(4)], [col(2), row(3)])] #ring 2

## Zeichenfunktionen
`init_canvas()` erstellt den Aufbau des Spielfeldes. Dies erfolgt mit Hilfe von ipycanvas. Das Spielbrett (`canvas`) besteht aus drei übereinanderliegenden Ebenen. Ebene 0 ist der Hintergrund, ein beiges Quadrat. Ebene 1 beinhaltet die schwarzen Striche und Punkte auf dem Spielfeld. Diese setzen sich aus drei schwarzen Quadraten, vier horizontalen bzw. waagrechten Linien und 24 Punkten an den Kreuzungen der Linien, die die möglichen Positionen der Steine markieren, zusammen. Dabei wird bei der Erstellung mit relativen Positionsangaben gearbeitet, um das Spielfeld einfach skalieren zu können. In der Ebene 2 liegen schließlich die Spielsteine. 

In [ ]:
def init_canvas():
    #canvas[Hintergrund, Linien, Steine]
    canvas = MultiCanvas(3, width = BOARD_SIZE, height = BOARD_SIZE)

    # Hintergrund
    canvas[0].fill_style = '#ffffcc'
    canvas[0].fill_rect(0, 0, BOARD_SIZE)

    # Strichstärke
    canvas[1].line_width = 5

    # Quadrate
    canvas[1].stroke_rect(BOARD_SIZE * col(0), BOARD_SIZE * row(0), BOARD_SIZE * (1 - row(0) - col(0))) #ring 0
    canvas[1].stroke_rect(BOARD_SIZE * col(1), BOARD_SIZE * row(1), BOARD_SIZE * (1 - row(1) - col(1))) #ring 1
    canvas[1].stroke_rect(BOARD_SIZE * col(2), BOARD_SIZE * row(2), BOARD_SIZE * (1 - row(2) - col(2))) #ring 2

    # Mittelinien
    canvas[1].begin_path()
    canvas[1].move_to(BOARD_SIZE * col(3), BOARD_SIZE * row(0)) #oben
    canvas[1].line_to(BOARD_SIZE * col(3), BOARD_SIZE * row(2))
    canvas[1].move_to(BOARD_SIZE * col(6), BOARD_SIZE * row(3)) #rechts
    canvas[1].line_to(BOARD_SIZE * col(4), BOARD_SIZE * row(3))
    canvas[1].move_to(BOARD_SIZE * col(3), BOARD_SIZE * row(6)) #unten
    canvas[1].line_to(BOARD_SIZE * col(3), BOARD_SIZE * row(4))
    canvas[1].move_to(BOARD_SIZE * col(0), BOARD_SIZE * row(3)) #links
    canvas[1].line_to(BOARD_SIZE * col(2), BOARD_SIZE * row(3))
    canvas[1].stroke()

    # Punkte (außen, mitte, innen)
    for ring in POSITIONS:
        for x, y in ring:
            canvas[1].fill_arc(BOARD_SIZE * x, BOARD_SIZE * y, DOT_RADIUS, 0, 360)
    return canvas

Die Funktion `draw_piece()` zeichnet an einer zu übergebenen Position einen Spielstein für den eingegebenen Spieler. Dieser besteht aus einem mit der Spielerfarbe gefüllten Kreis als Hintergrund und zwei Innenringen um die Steine realistischer aussehen zu lassen.

In [ ]:
def draw_piece(canvas, ring, cell, player):
    #Hintergrund
    canvas[2].fill_style = COLOUR[player]
    canvas[2].fill_arc(BOARD_SIZE * POSITIONS[ring][cell][0], BOARD_SIZE * POSITIONS[ring][cell][1], PIECE_RADIUS, 0, 360)
    
    #Ringe
    canvas[2].stroke_style = 'silver' if player == 1 else 'chocolate'
        
    canvas[2].stroke_arc(BOARD_SIZE * POSITIONS[ring][cell][0], BOARD_SIZE * POSITIONS[ring][cell][1], PIECE_RADIUS,       0, 360)
    canvas[2].stroke_arc(BOARD_SIZE * POSITIONS[ring][cell][0], BOARD_SIZE * POSITIONS[ring][cell][1], PIECE_RADIUS * 0.7, 0, 360)
    canvas[2].stroke_arc(BOARD_SIZE * POSITIONS[ring][cell][0], BOARD_SIZE * POSITIONS[ring][cell][1], PIECE_RADIUS * 0.3, 0, 360)

Mit Hilfe von `highlight_positions()` werden alle Positionen aus einer Liste mit einem transparenten Kreis markiert. Falls keine andere Farbe übergeben wird, wird die in den Konstanten definierte default-Farbe verwendet. Diese Funktion kann verwendet werden, um die schlagbaren Steine des Gegners oder die möglichen Zugpositionen zu bestimmen.

In [ ]:
def highlight_positions(canvas, highlight_positions, colour = COLOUR_HINT):
    with ipycanvas.hold_canvas(canvas):
        for (ring, cell) in highlight_positions:
            canvas[2].fill_style = colour
            canvas[2].global_alpha = TRANSPARENCY_HINT
            canvas[2].fill_arc(BOARD_SIZE * POSITIONS[ring][cell][0], BOARD_SIZE * POSITIONS[ring][cell][1], PIECE_RADIUS, 0, 360)

Die Funktion `show_start_button()` zeichnet auf dem Spielfeld ein Rechteck, dass das Wort "Start" enthält und optisch einem Button ähnelt.

In [ ]:
def show_start_button(canvas):
    with ipycanvas.hold_canvas(canvas):
        # Hintergrund
        canvas[2].clear()
        canvas[2].fill_style = 'black'
        canvas[2].global_alpha = TRANSPARENCY_HINT
        canvas[2].fill_rect(0, 0, BOARD_SIZE, BOARD_SIZE)
        
        # Button
        canvas[2].global_alpha = TRANSPARENCY_DEFAULT
        canvas[2].shadow_color = 'black'
        canvas[2].shadow_offset_x = 5
        canvas[2].shadow_offset_y = 5
        canvas[2].shadow_blur = 5
        canvas[2].fill_style = 'silver'
        canvas[2].fill_rect(BOARD_SIZE/3, (BOARD_SIZE/5)*2, BOARD_SIZE/3, height=BOARD_SIZE/5)
        
        canvas[2].shadow_offset_x = 0
        canvas[2].shadow_offset_y = 0
        canvas[2].shadow_blur = 0
        
        canvas[2].stroke_style = 'black'
        canvas[2].stroke_rect(BOARD_SIZE/3, (BOARD_SIZE/5)*2, BOARD_SIZE/3, height=BOARD_SIZE/5)
        
        # Text
        canvas[2].fill_style = 'black'
        canvas[2].font = '30px Arial'
        canvas[2].fill_text('Start', BOARD_SIZE/2 *0.83, BOARD_SIZE/2*1.05)

`update_canvas()` zeichnet die obere Ebene des Spielbretts nach jeder Veränderung neu. So werden Positionsänderungen oder Hinweise zu möglichen Zielfeldern und schlagbaren Steinen sichtbar. Zusätzlich wird hier die Aktualisierung der Statusanzeige angestoßen.
Durch den optionalen Parameter `message` ist es möglich die Statusausgabe zu überschreiben. Fals das Spiel gerade pausiert ist, wird außerdem ein Start-Button eingeblendet.

In [ ]:
def update_canvas(game, message = ''):
    gamemode_side_by_side = widgets.HBox([gamemode_label, gamemode_toggle])
    one_below_other       = widgets.VBox([gamemode_side_by_side, turn_label, status_label, pieces_status_label, pieces_player_one, pieces_player_two, restart_button])
    side_by_side          = widgets.HBox([game.canvas, one_below_other])
    display(side_by_side)

    with ipycanvas.hold_canvas(game.canvas):
        game.canvas[2].clear()
        game.canvas[2].global_alpha = TRANSPARENCY_DEFAULT
        for ring in range(3):
            for cell in range(8):
                value = game.state[1][ring][cell]
                if value in PLAYER: draw_piece(game.canvas, ring, cell, value) 
        update_status_widgets(game, message)    
    if game.pause == True:
        show_start_button(game.canvas)

## Die Klasse Game - Status des Spielfelds
Der Status speichert die aktuelle Situation des Spielfeldes. Die Klasse beinhaltet mehrere Variablen. `state` gibt wie in der Definition beschrieben an, ob und wenn ja welcher Stein sich an einer Position auf dem Spielfeld befindet. Dabei gilt:
- 0 = kein Stein
- 1 = weißer Stein
- 2 = brauner Stein

Außerdem beinhaltet die Variable Angaben über die Steine, die die beiden Spieler nocht setzen können.
`current_player` gibt an, welcher Spieler, 1 oder 2, gerade am Zug ist. `canvas` ist ein ipycanvas-Objekt, dass mittels `init_canvas()` erzeugt wird. Es hadelt sich dabei um die visuelle Ausgabe des Spielfeldes. In `number_pieces_to_remove` wird gespeichert, wie viele Steine der aktuelle Spieler aufgrund von Mühlen noch entfernen darf. `selected_piece` ist ein Tupel, das die Position eines ausgewählten Steins speichert. Dieses Tupel hat die Form `(ring, cell)`. Dies ist notwendig für Zug- und Springvorgänge. Ist die Variable ungleich `None`, befindet sich der Spieler gerade in einer Zug- oder Sprungaktion. Ist die Variable  belegt, so hat der Spieler bereits den zu versetzenden Stein ausgewählt und muss noch auf das Zielfeld klicken. Die Position ist in Phase 2 des Spiels relevant, um valide Zielfelder zu ermitteln. In der Phase 1 kann es vorkommen, dass kein Stein gesetzt wird, weil kein passendes Feld augewählt wurde, um zu verhindern, dass der Spieler trotzdem wechselt, wird die Variable `do_not_change` auf True gesetzt und diese vor Spielerwechsel abgefragt. Das Gleiche geschieht in den Phasen 2 und 3, falls kein passender Stein ausgewählt wird. `game_over`ist False, solange das Spiel noch läuft. Die Variable `pause` ist nur direkt nach der Initialisierung `True`. Beim ersten Klicken auf das Spielfeld, dem Starten des Spiels, wird sie auf `False` gesetzt. Dies ist notwendig, um die Berechnung des nächsten Zustands anzustoßen, falls der erste Zug durch einen Such-Algorithmus erfolgt.
Ein Spieler kann entweder eine menschliche Person oder ein Such-Algorithmus sein. Als Algorithmen sind Minimax und Alpha-Beta-Pruning implementiert. Standardmäßig sind Spieler reale Personen. Um was es sich handelt, ist in den Variablen `mode_player_one` und `mode_player_two` gespeichert.

Die `__init__()` Funktion setzt die Werte der Game-Klasse. Dabei kann falls notwendig der aktuelle Spieler übergeben werden, falls beispielsweise Spieler 2 starten soll. Standardmäßig wird als Zustand der Startzustand übergeben, wobei davon zuvor eine Deepcopy angefertigt werden muss. Dies ist notwendig, da ansonsten die Refernz auf `START_STATE` übergeben wird und die Konstante somit im Laufe des Spiel verändert werden würde. Außerdem kann angegeben werden, dass die Spieler statt manuell zu spielen einen Such-Algorithmus zur Bestimmung des nächsten Spiel-Zustands verwenden. Hierfür existieren die Variablen `mode_player_one` und `mode_player_two` die die gewünschte Option für Spieler 1 bzw. 2 angeben und folgende Werte annehmen können:
- None = Der Spieler spielt manuell
- 'minimax' = Der Spieler nutzt das Suchverfahren Minimax
- 'alpha-beta' = Der Spieler nutzt das Suchverfahren Alpha-Beta-Pruning

Nach der Initialisierung der Variablen wird noch festgelegt, dass bei einem Klick auf das Spielfeld die klasseneigene Funktion `play_game()` aufgerufen wird. Diese wird gleich noch definiert.

Mit Hilfe der Funktion `__str__()` kann der Status später mit dem Befehl `print(Game)` formatiert ausgegeben werden. Dies ist besonders zum Überprüfen des aktuellen Spielstandes hilfreich.

In [ ]:
class Game():
    def __init__(self, current_player = 1, mode_player_one = None, mode_player_two = None):
        self.state          = copy.deepcopy(START_STATE)
        self.current_player = current_player
        self.canvas         = init_canvas()
        self.number_pieces_to_remove = 0
        self.selected_piece = (None, None)
        self.do_not_change  = False
        self.game_over      = False
        self.pause          = True
        
        # TODO: mode_player_one/mode_player_two darf nur "alpha-beta" oder "minimax" sein, dies könnte/sollte schon bei der Initialisierung abgefragt werden
        self.mode_player_one = mode_player_one 
        self.mode_player_two = mode_player_two
        
        self.canvas[2].on_mouse_down(self.play_game)
    
    def __str__(self):
        return "state: " + str(self.state) \
            + "\nmode player 1: " + str(self.mode_player_one) \
            + "\nmode player 2: " + str(self.mode_player_two) \
            + "\ncurrent player: " + str(self.current_player) \
            + "\nnumber pieces to remove: " + str(self.number_pieces_to_remove) \
            + "\nselected piece: " + str(self.selected_piece) \
            + "\ndo not change: " + str(self.do_not_change) \
            + "\ngame over: " + str(self.game_over) \
            + "\npause: " + str(self.pause)

Die Funktion `play_game()` der Klasse `Game` steuert das Spiel. Sie wird später aufgerufen, wenn der Spieler irgendwo hin klickt. Wenn das Spiel noch nicht vorbei ist, wird überprüft, ob das Spiel gerade pausiert ist. Das ist beim ersten Klicken der Fall. Ist dem so, wird  die Spielmodusauswahl deaktiviert und überprüft, ob ein Algorithmus als erstes dran ist. Ist dem so, führt dieser seinen Zug durch. Ansonsten wird das Spielbrett lediglich aktualisiert, um dem manuellen Spieler zu sagen, was er zu tun hat. Bei jedem weiteren Klick wird überprüft, ob dieser an einer Positionen, also im Bereich um einen schwarzen Punkt, erfolgt ist. Wenn ja, kann eine der folgenden Situationen eintreten, wobei die erste aufgelistete Situation Priorität hat:
- **Es müssen noch Steine entfernt werden**  - Aufgrund von Mühlen die zuvor geschlossen wurden, dürfen noch Steine entfernt werden. Der zuvor geklickte Stein wird der Funktion `remove_piece()` übergeben, die ihn vom Spielfeld löscht.
- **Der Spieler befindet sich in Phase 1** - Auf die ausgewählte Position wird mit Hilfe von `place_piece()` ein Stein gesetzt.
- **Es wurde zuvor kein Stein ausgewählt** - Da sich der Spieler wie zuvor überprüft nicht in Phase 1 und somit in der Zug- oder Springphase befindet, und bisher kein Startfeld ausgewählt wurde, kann dies nun geschehen. Dazu wird die Funktion `select_piece()` aufgerufen.
- **Es wurde ein Stein ausgewählt** - Da sich der Spieler in der Zug- oder Springphase befindet, handelt es sich bei dem ausgewählten Feld um ein Zielfeld, auf dass der zuvor ausgewählte Stein mittels `move_piece()` bewegt wird.

Falls nach abarbeiten der Situationen noch Steine zu entfernen sind, werden diese markiert. Außerdem wird überprüft, ob das Spiel zu Ende ist. Falls sich der Spieler nicht in einem Zwischenzustand von Phase 2 oder 3 (Startfeld ausgewählt, Zielfeld noch nicht bekannt) befindet, keine Steine mehr entfernt werden müssen, das Spiel nicht zu Ende ist und vorher alle korrekt gelaufen ist, ist der nächste Spieler an der Reihe und das Spielfeld wird aktualisiert. `algo_step()` wird aufgerufen, um zu überprüfen, ob der nächste Zug ein Zug der KI ist, der gegebenenfalls ausgeführt werden muss.

In [ ]:
def play_game(self, x, y):
    if self.game_over: return
    
    if self.pause:
        self.pause = False
        gamemode_toggle.disabled = True
        if (self.current_player == 1 and self.mode_player_one != None) or (self.current_player == 2 and self.mode_player_two != None):
            self.algo_step()
        else:
            update_canvas(self)
        return 
    
    phase = get_player_phase(self.state, self.current_player)
    piece = self.get_clicked_piece(x, y)

    # Situationen überprüfen
    if piece == None:
        return
    if self.number_pieces_to_remove > 0:
        self.remove_piece(piece)
    elif phase == 1:
        self.place_piece(piece)
    elif self.selected_piece != (None, None):
        self.move_piece(piece)
    elif phase == 2 or phase == 3:
        self.select_piece(piece)
    else:
        change_status_label('Du befindest Dich anscheinend in keiner gültigen Spielphase. Irgendwas ist schief gegangen... Sorry!')

    if self.number_pieces_to_remove > 0:
        update_canvas(self)
        highlight_positions(self.canvas, get_opponent_beatable_pieces(self.state[1], self.current_player), colour = COLOUR_OPPONENT)

    if finished(self.state): self.game_over = True

    if self.selected_piece == (None, None) and not self.number_pieces_to_remove > 0 and not self.game_over and not self.do_not_change:
        self.current_player = opponent(self.current_player) # Spielerwechsel, wenn nicht noch eine Aktion ausgeführt werden muss
        update_canvas(self)

    self.algo_step()
        
Game.play_game = play_game
del play_game

Die Funktion `get_clicked_piece()` der Klasse `Game` überprüft, ob die übergebenen x- und y- Koordinaten auf einer validen Position, also einem Spielfeldpunkt liegen. Wenn ja, wird diese Position in der Form `(ring, cell)` zurückgegeben, ansonsten `None`. Zur Ermittlung wird über alle möglichen Positionen iteriert und geprüft, ob die Klick-Koordinaten in einem Pufferbereich rund um den Mittelpunkt der Position liegen. Ist dies für beide Koordinaten der Fall, wurde der angeklickte Punkt gefunden.

In [ ]:
def get_clicked_piece(self, x, y):
    piece = None
    #TODO: Schleife abbrechen, sobald piece gesetzt wurde
    for ring in POSITIONS:
        for pos in ring:
            check_x = pos[0] * BOARD_SIZE - PIECE_RADIUS < x < pos[0] * BOARD_SIZE + PIECE_RADIUS
            check_y = pos[1] * BOARD_SIZE - PIECE_RADIUS < y < pos[1] * BOARD_SIZE + PIECE_RADIUS
            if check_x and check_y:
                piece = (POSITIONS.index(ring), ring.index(pos))
    return piece
Game.get_clicked_piece = get_clicked_piece
del get_clicked_piece

In Phase 1 wird durch `place_piece()` der Klasse `Game` überprüft, ob das angeklickte Feld noch frei ist. Wenn ja, wird dort ein Stein des Spielers positioniert und die eigenen Steinanzahl in `remaining` um 1 reduziert. Um zu überprüfen, ob bei der Aktion neue Mühlen entstanden sind, wird vor und nach dem Setzen des Steins alle Mühlen gesucht. Anschließend wird geprüft, ob und wenn ja, wie viele, neue Mühlen sich in der Menge der Mühlen nach dem Setzen des Steins befinden. Dies ist notwendig, um zu ermitteln, wie viele Steine des Gegners entfernt werden dürfen.

In [ ]:
def place_piece(self, piece):
    remaining, board = self.state
    if piece in empty_positions(board):
        self.do_not_change = False
        ring, cell = piece
        mills_before = find_mills(board, self.current_player)
        board[ring][cell] = self.current_player
        remaining[self.current_player - 1] -= 1 
        mills_after = find_mills(board, self.current_player)
        self.number_pieces_to_remove = count_new_mills(mills_before, mills_after)
        update_canvas(self)
    else:
        self.do_not_change = True
        change_status_label('Du kannst Deinen Stein nur auf ein leeres Feld setzen. Bitte probiere es erneut!')
Game.place_piece = place_piece
del place_piece

`remove_piece()` der Klasse `Game` entfernt den übergebenen Stein vom Spielfeld, falls es erlaubt ist, diesen zu entfernen. Dazu  werden alle schlagbaren Steine des Gegners ermittelt und überprüft, ob der angeklickte Stein dazu gehört. Ist dies der Fall, wird der Stein enfernt, indem an dieser Stelle eine `0` eingetragen wird. Andernfalls wird eine Fehlermeldung geworfen und der Spieler kann einen anderen Stein zum Entfernen auswählen. Falls noch mehr Steine entfernt werden müssen, werden die möglichen Positionen am Ende markiert.

In [ ]:
def remove_piece(self, piece):
    _, board = self.state
    ring, cell = piece
    beatable_pieces = get_opponent_beatable_pieces(board, self.current_player)
    if piece in beatable_pieces:
        board[ring][cell] = 0
        self.number_pieces_to_remove -= 1
        update_canvas(self)
    else:
        change_status_label('Du kannst diesen Stein nicht entfernen. Bitte probiere es erneut!')
    if self.number_pieces_to_remove > 0:
        highlight_positions(self.canvas, get_opponent_beatable_pieces(board, self.current_player), colour = COLOUR_OPPONENT)
Game.remove_piece = remove_piece
del remove_piece

`move_piece()` der Klasse `Game` bewegt den zuvor ausgewählten Stein, falls zulässig, auf die übergebene Position. Das Startfeld ist dabei in der Variable `selected_piece` gespeichert. Damit das Zielfeld valide ist, muss es in der Liste sein, die durch `next_positions()` ausgehend vom ausgewählten Stein ermittelt werden. Ist dies der Fall, wird das alte Feld auf `0` gesetzt und das Zielfeld mit dem Wert des aktuellen Spielers belegt. Um zu überprüfen, ob bei der Aktion neue Mühlen entstanden sind, wird vor und nach dem Versetzen des Steins alle Mühlen gesucht. Anschließend wird geprüft, ob und wenn ja, wie viele, neue Mühlen sich in der Menge der Mühlen nach dem Bewegen des Steins befinden. Dies ist notwendig, um zu ermitteln, wie viele Steine des Gegners entfernt werden dürfen.

In [ ]:
def move_piece(self, piece):
    _, board = self.state
    start_ring, start_cell = self.selected_piece
    goal_ring,  goal_cell  = piece
    possible_positions = next_positions(self.state, self.current_player, start_ring, start_cell)
    if piece in possible_positions:
        mills_before = find_mills(board, self.current_player)
        board[start_ring][start_cell] = 0
        board[goal_ring] [goal_cell]  = self.current_player
        mills_after = find_mills(board, self.current_player)
        self.number_pieces_to_remove = count_new_mills(mills_before, mills_after)
        self.selected_piece = (None, None)
        update_canvas(self)
    else:
        change_status_label('Du kannst Deinen Stein nicht auf dieses Feld setzen. Bitte probiere es erneut!')
Game.move_piece = move_piece
del move_piece

Falls sich der Spieler in der Zug- oder Springphase befindet, und noch kein Startfeld ausgewählt wurde, wird `select_piece()` der Klasse `Game` aufgerufen. Diese Funktion überprüft, ob der übergebene Stein ein eigener Stein ist. Wenn ja wird er in der Variable `selected_piece` gespeichert und ausgehend davon mögliche Zielfelder berechnet, um diese hervorheben zu können. Nach erfolgreicher Durchführung befindet sich der Spieler in einem Zwischenzustand, da das Zielfeld noch nicht bekannt ist.

In [ ]:
def select_piece(self, piece):
    if piece in player_pieces(self.state[1], self.current_player):
        self.do_not_change = False
        self.selected_piece = piece
        change_status_label('Zielort für den eigenen Stein auswählen.')
        ring, cell = piece
        possible_positions = next_positions(self.state, self.current_player, ring, cell)
        highlight_positions(self.canvas, possible_positions)
    else:
        self.do_not_change = True
        change_status_label('Das ist nicht Dein Stein. Bitte probiere es erneut!')
Game.select_piece = select_piece
del select_piece

Die Funktion `algo_step()` der Klasse `Game` führt einen Zug aus, falls der aktuelle Spieler ein Algorithmus ist. Als Algorithmen stehen Alpha-Beta-Pruning und Minimax zur Verfügung. Dazu wird zunächst überprüft, ob nicht vielleicht das Spiel schon beendet ist. In diesem Fall würde gar nichts geschehen. Andernfalls wird geprüft, um welchen Algorithmus es sich handelt. Anschließend wird die entsprechende Funktion aufgerufen, die den nächsten Zustand berechnet und ausgibt. Dies ist entweder `alpha_beta_pruning()` oder `minimax()`. Der Zustand des Spiels wird auf den berechneten Zustand gesetzt und es wird geprüft, ob das Spiel mit dem Zug beendet wurde. Der Spieler wechselt und das Spielbrett wird aktualisiert. Falls das Spiel noch nicht beendet ist, wird die Funktion nochmal aufgerufen, da der nächste Spieler ebenfalls ein Such-Algorithmus sein könnte.

In [ ]:
def algo_step(self):
    #TODO: #91: Es kann passieren, dass ab einem Punkt immer nur zwischen zwei Zuständen gewechselt wird. Dies führt zu einem endlosen Spiel.
    if self.game_over: return
    if (self.current_player == 1 and self.mode_player_one == "alpha-beta") or (self.current_player == 2 and self.mode_player_two == "alpha-beta"):
        change_status_label("Der Computer rechnet noch... Ein Moment bitte.")
        _, self.state = alpha_beta_pruning(self.state, self.current_player)
        if finished(self.state): self.game_over = True
        self.current_player = opponent(self.current_player)
        update_canvas(self)        
        if not self.game_over: self.algo_step()
    elif (self.current_player == 1 and self.mode_player_one == "minimax") or (self.current_player == 2 and self.mode_player_two == "minimax"):
        change_status_label("Der Computer rechnet noch... Ein Moment bitte.")
        _, self.state = minimax(self.state, self.current_player)
        if finished(self.state): self.game_over = True
        self.current_player = opponent(self.current_player)
        update_canvas(self)        
        if not self.game_over: self.algo_step()
Game.algo_step = algo_step
del algo_step

## Die Statusanzeige - Initialisierung und Funktionen der Status-Widgets und des Restart-Button

Die Methode `restart()` wird durch den restart_button aufgerufen und setzt den Status des Spieles wieder zurück, indem eine neue Instanz der Klasse `Game` erzeugt und das Spielfeld anschließend aktualisiert wird. Außerdem wird die Spielmodusauswahl aktiviert.

In [ ]:
def restart(b):
    gamemode_toggle.disabled = False
    gamemode_toggle.value = 'Spieler vs Spieler'
    global game
    game = Game()
    clear_output(wait = True)
    update_canvas(game)

Die Methode `change_gamemode()` ist der handler für das game_toggle-Widget und ändert den Spielmodus nach dem aktuellen Wert des gamemode_toggle-Widget. 

In [ ]:
def change_gamemode(change):
    global game
    gamemode = gamemode_toggle.value
    if gamemode == 'Spieler vs Spieler':
        game.mode_player_one = None 
        game.mode_player_two = None
    elif gamemode == 'Spieler vs α-β':
        game.mode_player_one = None 
        game.mode_player_two = 'alpha-beta'
    elif gamemode == 'α-β vs Minimax':
        game.mode_player_one = 'alpha-beta'
        game.mode_player_two = 'minimax'

An dieser Stelle findet die Initialisierung der unterschiedlichen Statuselemente statt.
-  Das `turn_label` ist ein Label, das den Spieler anzeigt, der aktuell an der Reihe ist
-  Das `status_label` ist für weitere Statusanzeigen vorgesehen, wie die Aufforderung an den aktuellen Spieler, im Fall einer Mühle, einen Stein des Gegners zu entfernen.
-  Das `pieces_status_label` zeigt je nach Spielphase entweder die Anzahl der Steine, die noch gesetzt werden können (Phase 1), oder die noch übrig sind (Phase 2 und 3). 
-  Der `restart_button` ruft die Methode `restart()` auf und setzt somit das Spiel auf den Startzustand zurück
-  `pieces_player_one` und `pieces_player_two` zeigen jeweils einen horizontalen Balken, der in Phase 1 nach jedem gesetzten Stein kleiner wird, bis keiner mehr vorhanden ist. In Phase 2 und 3 ist die Auslenkung des Balkens von der Anzahl der Steine des jeweiligen Spielers abhängig. Somit ist schnell erkennbar, welcher Spieler gerade vorne liegt.
- Die ToggleButtons `gamemode_toggle` ermöglichen die Auswahl des Spielmodus (Player versus Player / Player versus Algorithmus / Algorithmus versus Algorithmus)
- Das `gamemode_label` ist das Label für die ToggleButtons

In [ ]:
turn_label          = widgets.Label(value = 'X ist an der Reihe')
status_label        = widgets.Label(value = 'Status:')
pieces_status_label = widgets.Label(value = 'Setzbare Steine:')

restart_button      = widgets.Button(description = 'Neustart')
restart_button.on_click(restart)

pieces_player_one   = widgets.FloatProgress(value = 9, max = 9, min = 0, description = PLAYER_COLOUR[0], style={'bar_color': 'silver'})
pieces_player_two   = widgets.FloatProgress(value = 9, max = 9, min = 0, description = PLAYER_COLOUR[1], style={'bar_color': COLOUR[2]})

gamemode_toggle     = widgets.ToggleButtons(
                        options     = ['Spieler vs Spieler', 'Spieler vs α-β', 'α-β vs Minimax'],
                        disabled    = False,
                        tooltips    = ['Spieler versus Player', 'Spieler versus Alpha-Beta-Algorithm', 'Alpha-Beta-Algorithm versus Minimax-Algorithm']
                    )
gamemode_toggle.observe(change_gamemode, 'value')
gamemode_label      = widgets.Label(value = 'Spielmodus: ')


Die Methode `change_status_label()` nimmt einen beliebigen Status-Text entgegen und setzt diesen mit dem Präfix 'Status: ' als Text des `status_label`.

In [ ]:
def change_status_label(message):
    status_label.value = 'Status: ' + message

Die Methode update_status_widgets() aktualisiert die unterschiedlichen Statusanzeigen passend zur aktuellen Spielphase und den zuletzt getätigten bzw. erwarteten Aktionen.

In [ ]:
def update_status_widgets(game, message):
    remaining, board = game.state
    player = game.current_player
    turn_label.value = PLAYER_COLOUR[player - 1] + ' ist an der Reihe.'
    
    phase = get_player_phase(game.state, player)
    if phase == 1: change_status_label('Bitte einen Stein auf dem Spielfeld platzieren.')
    if phase == 2: change_status_label('Bitte einen Stein zum Ziehen auswählen.')
    if phase == 3: change_status_label('Bitte einen Stein zum Springen auswählen.')
        
    if game.pause: change_status_label('Bitte oben den Spielmodus auswählen und anschließend auf "Start" klicken.')
        
    if remaining[0] == remaining[1] == 0:
        pieces_status_label.value = 'Verbleibende Steine:'
        pieces_player_one.value = count_player_pieces(board, 1)
        pieces_player_two.value = count_player_pieces(board, 2)
    else:
        pieces_status_label.value = 'Setzbare Steine:'
        pieces_player_one.value = remaining[0]
        pieces_player_two.value = remaining[1]
         
    if message != '': change_status_label(message)
    if finished(game.state) and utility(game.state, player) == 1: change_status_label(PLAYER_COLOUR[player - 1] + ' hat gewonnen!')
    if finished(game.state) and utility(game.state, player) == -1: change_status_label(PLAYER_COLOUR[opponent(player) - 1] + ' hat gewonnen!')
    if game.number_pieces_to_remove == 1: change_status_label('Mühle! Bitte noch 1 Stein des Gegners entfernen.') 
    if game.number_pieces_to_remove > 1:  change_status_label('Mühle! Bitte noch ' + str(game.number_pieces_to_remove) + ' Steine des Gegners entfernen.') 

## Spielen

Schließlich kann das Spielfeld erzeugt und ein Spiel gestartet werden. Dabei wird bei jedem Mausklick auf das Spielfeld `play_game()` mit den Koordinaten des Klicks aufgerufen.

In [ ]:
game = Game()
update_canvas(game)